In [1]:
!ls

Basics.ipynb  chinook.db  chinook-unmodified.db


In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [3]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


* finding the number of tracks sold
* in the USA
* for each Genre

In [4]:
%%sql
WITH USA_genres AS (
    SELECT
        cust.country,
        il.quantity,
        il.track_id,
        g.genre_id,
        g.name genre_name
    FROM customer cust
    INNER JOIN invoice inv ON inv.customer_id == cust.customer_id
    INNER JOIN invoice_line il ON inv.invoice_id == il.invoice_id
    INNER JOIN track t ON t.track_id == il.track_id
    INNER JOIN genre g ON t.genre_id == g.genre_id
    WHERE country == 'USA'
)

SELECT
    genre_name,
    SUM(quantity) USA_total_tracks_sold,
    CAST(SUM(quantity) AS FLOAT) / (
        SELECT SUM(quantity)
        FROM USA_genres
    ) percent_sold

FROM USA_genres
GROUP BY genre_name
ORDER BY USA_total_tracks_sold DESC
LIMIT 10;

Done.


genre_name,USA_total_tracks_sold,percent_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


Based on the track sales shown above, the most stable foreign artists to bring to the USA are:
* Mayhem from Norway (Rock)
* Sex Pistols from England (Punk)
* Motörhead from England (Metal)

These genre's may seem similiar and initial concerns would be that they would cannabalise each other's market share but, the combined market share of these 3 genres is ~77% of the music industry. There will be some people who're interested in these Artists.

In [5]:
%%sql
--for each sales support agent
--find the total dollar amount of sales

--get the mean of the toal dollar amount sold
--get employee deviation from the mean
--sort by absolute value of deviation

/*
SELECT
    cust.support_rep_id,
    (e1.first_name || " " || e1.last_name) name,
    SUM(inv.total) total_sales
FROM invoice inv
LEFT JOIN customer cust ON cust.customer_id == inv.customer_id
LEFT JOIN employee e1 ON cust.support_rep_id == e1.employee_id
GROUP BY support_rep_id
ORDER BY total DESC;
*/
SELECT 
    cust.support_rep_id, 
    (e1.first_name || " " || e1.last_name) name,
    SUM(inv.total) total_sales
FROM employee e1
INNER JOIN customer cust ON cust.support_rep_id == e1.employee_id
INNER JOIN invoice inv ON cust.customer_id == inv.customer_id
GROUP BY support_rep_id
ORDER BY total DESC;

Done.


support_rep_id,name,total_sales
5,Steve Johnson,1393.9200000000028
3,Jane Peacock,1731.510000000004
4,Margaret Park,1584.0000000000032


There are only 4 support representives??

There isn't much of a difference between the top seller & bottom seller.

In [6]:
%%sql
-- for each country calculate:
    
-- total number of customers
-- total value of sales
-- average value of sales per customer  (invoice total for ea customer / COUNT(customer))
-- average order value (AVG across all invoices in that country)

-- customer, invoice


-- group by customer, avg their sales
-- group by country

DROP VIEW IF EXISTS cust_table;
CREATE VIEW cust_table AS
    SELECT
        cust.country,
        COUNT(DISTINCT(cust.customer_id)) customer_count,
        SUM(inv.total) total_sales,
        SUM(inv.total) / COUNT(DISTINCT(cust.customer_id)) money_spent_per_customer,
        AVG(inv.total) avg_order_value
    FROM customer cust
    INNER JOIN invoice inv ON cust.customer_id == inv.customer_id
    GROUP BY country;

WITH small_country_as_other AS(
    SELECT
        country,
        customer_count,
        total_sales,
        money_spent_per_customer,
        avg_order_value,
        CASE
            WHEN customer_count == 1 THEN "Other"
            ELSE cust_table.country
            END AS other_country,
        CASE
            WHEN customer_count == 1 THEN 1
            ELSE 0
            END AS has_one_cust
    FROM cust_table
)

SELECT
    other_country AS country_other,
    SUM(customer_count) AS customer_count,
    SUM(total_sales) AS total_sales,
    AVG(money_spent_per_customer) AS money_spent_per_customer,
    AVG(avg_order_value) avg_order_value
FROM small_country_as_other
GROUP BY country_other
ORDER BY has_one_cust ASC, total_sales DESC;

Done.
Done.
Done.


country_other,customer_count,total_sales,money_spent_per_customer,avg_order_value
USA,13,1040.4899999999998,80.0376923076923,7.942671755725189
Canada,8,535.5900000000001,66.94875000000002,7.047236842105265
Brazil,5,427.68000000000006,85.53600000000002,7.0111475409836075
France,5,389.0699999999999,77.81399999999998,7.781399999999998
Germany,4,334.62,83.655,8.161463414634147
Czech Republic,2,273.24000000000007,136.62000000000003,9.108000000000002
United Kingdom,3,245.52,81.84,8.768571428571429
Portugal,2,185.13000000000002,92.56500000000001,6.383793103448276
India,2,183.14999999999998,91.57499999999999,8.72142857142857
Other,15,1094.9399999999998,72.996,7.445071062271063


In [7]:
%%sql

-- how many invoices were album purchases
-- by raw number
-- by percentage

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


Album purchases account for 18.6% of purchases. Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.